In [1]:
import os

import requests
import json
import pandas as pd
import numpy as np
from dotenv import load_dotenv

load_dotenv()
KAMIS_CERT_KEY = os.getenv('KAMIS_CERT_KEY')
KAMIS_CERT_ID = os.getenv('KAMIS_CERT_ID')
CATEGORY_CODES = ['100', '200', '300', '400']

In [2]:
# 18-01-01 ~ 23-12-31
# 사용할 작물별로 그날의 평균값
# 10일 단위로 최대 최소 가격
# 10일 단위 전체 평균 가격

In [38]:
# api 호출
def get_daily_price(date: str, category_code: str) -> json:
  headers = {'Content-Type': 'application/json', 'charset': 'UTF-8', 'Accept': '*/*'}
  request_url = 'http://www.kamis.or.kr/service/price/xml.do?action=dailyPriceByCategoryList'
  params = {
    'p_cert_key':KAMIS_CERT_KEY,
    'p_cert_id':KAMIS_CERT_ID,
    'p_item_category_code':category_code,
    'p_regday':date,
    'p_returntype':'json'
  }
  
  for param in params.keys():
    request_url += f"&{param}={params[param]}"

  try:
    return requests.get(request_url, headers=headers).json()
  except Exception as e:
    print(e)

In [59]:
product_cost_list = []
for date in pd.date_range('2018-01-01', '2018-02-01').strftime('%Y-%m-%d'):
  # 하루단위 해당 작물 총 합
  product_acc_cost = {}
  for cat in CATEGORY_CODES:
    response = get_daily_price(str(date), cat)
    
    data = response['data']
    if type(data) == list:
      continue

    for item in response['data']['item']:
      item_code = int(item['item_code'])
      item_name = item['item_name']
      product_cost = item['dpr1'].replace(',','')
      
      if product_cost == '-':
        continue
  
      product_cost = int(product_cost)
      
      if product_acc_cost.get(item_code) is None:
        product_acc_cost[item_code] = {
          'total_cost':0,
          'total_count':0,
        }

      product_acc_cost[item_code]['total_cost'] = product_acc_cost[item_code]['total_cost'] + product_cost
      product_acc_cost[item_code]['total_count'] = product_acc_cost[item_code]['total_count'] + 1
  
  product_daily_avg_cost = {'date':date}
  
  for key in product_acc_cost.keys():
    acc_cost = product_acc_cost.get(key)
    product_daily_avg_cost[key] = acc_cost['total_cost'] / acc_cost['total_count']
    product_cost_list.append(product_daily_avg_cost)
  

In [61]:
count_per_ten_days = 0
for product_cost in product_cost_list:
  date = product_cost['date']
  index = (date.month - 1) * 3
  day = date.day
  first = 
  
  if day <= 10:
    
  

[{'date': '2018-01-02',
  111: 38300.0,
  112: 97400.0,
  141: 158476.0,
  142: 272800.0,
  143: 342800.0,
  144: 2840.0,
  151: 27200.0,
  152: 45600.0,
  211: 6350.0,
  212: 8450.0,
  213: 10600.0,
  214: 21100.0,
  215: 3680.0,
  221: 8050.0,
  223: 33138.833333333336,
  224: 18750.0,
  225: 9950.0,
  226: 22600.0,
  231: 7650.0,
  232: 17000.0,
  233: 8480.0,
  241: 630000.0,
  242: 33666.666666666664,
  243: 61200.0,
  244: 62975.0,
  245: 16950.0,
  246: 1770.0,
  247: 48700.0,
  252: 28687.5,
  253: 26700.0,
  255: 28100.0,
  256: 17500.0,
  257: 21400.0,
  258: 123800.0,
  422: 13800.0,
  312: 310800.0,
  313: 169333.33333333334,
  314: 230500.0,
  315: 12400.0,
  316: 9000.0,
  317: 7300.0,
  411: 31900.0,
  412: 31000.0,
  414: 32700.0,
  415: 19400.0,
  416: 28700.0,
  418: 20600.0,
  419: 30000.0,
  420: 23600.0,
  421: 54100.0,
  424: 63600.0,
  428: 44600.0},
 {'date': '2018-01-02',
  111: 38300.0,
  112: 97400.0,
  141: 158476.0,
  142: 272800.0,
  143: 342800.0,
  144: 

In [ ]:
product_cost_fluctuation_data = pd.DataFrame(columns=['YEAR', 'COUNT_PER_TEN_DAYS', 'PRODUCT_CODE', 'PRODUCT_NAME', 'MIN_PRODUCT_COST', 'MAX_PRODUCT_COST', 'AVG_PRODUCT_COST'])